In [19]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_facenet import FaceNet
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from ultralytics import YOLO 
import joblib

face_detector = YOLO("face_yolov8s.pt")
facenet_model = FaceNet()
classes = ["Aahad", "Ali", "Baba", "Bisma", "Damysha", "Daud", "Fahad", "Gujjar", "Sameed", "Saqib", "Unaiza", "Zunaira"]

# Load the saved classifier
classifier = joblib.load('classifier.pkl')
print("Classifier loaded successfully!")

Classifier loaded successfully!


In [20]:
def detect_and_crop_faces(image_path):
    """
    Detects and crops multiple face regions from an image using YOLOv8 face detection.
    Args:
        image_path (str): Path to the input image.
    Returns:
        List of PIL.Image: List of cropped face images or an empty list if no face is detected.
    """
    image = Image.open(image_path).convert("RGB")
    results = face_detector.predict(source=np.array(image), conf=0.5, iou=0.45, save=False, save_crop=False, device="cpu")
    
    # Initialize list to hold cropped faces
    cropped_faces = []
    
    # Process detection results
    if len(results[0].boxes) > 0:
        for box in results[0].boxes:
            # Extract bounding box coordinates
            coordinates = box.xyxy[0].cpu().numpy()
            x1, y1, x2, y2 = map(int, coordinates)
            # Crop the face and resize for FaceNet
            cropped_face = image.crop((x1, y1, x2, y2))
            cropped_faces.append(cropped_face.resize((160, 160)))  # Resize for FaceNet
    
    return cropped_faces

In [21]:
import cv2
import numpy as np

def predict_images(image_path):
    """
    Predicts whether the faces in the input image are 'aahad' or 'sameed'.
    Args:
        image_path (str): Path to the input image.
    Returns:
        List of str: Predicted class labels for each face, or an empty list if no faces are detected.
    """
    # Detect and crop multiple faces
    cropped_faces = detect_and_crop_faces(image_path)
    
    predictions = []
    
    # If no faces are detected, return a message
    if len(cropped_faces) == 0:
        return ["No face detected"]
    
    # Iterate over each cropped face
    for cropped_face in cropped_faces:
        # Convert PIL image to NumPy array
        cropped_face_array = np.array(cropped_face)
        
        # Resize image to FaceNet's expected input size (160x160)
        cropped_face_resized = cv2.resize(cropped_face_array, (160, 160))
        
        # Generate embedding for the cropped face
        embedding = facenet_model.embeddings([cropped_face_resized])[0].reshape(1, -1)
        
        # Predict using the trained SVM classifier
        prediction = classifier.predict(embedding)
        predictions.append(classes[prediction[0]])  # Append the predicted class label
    
    return predictions  # Returns a list of predicted class labels for all faces

In [23]:
# Example usage of the prediction function:
input_image_path = "./test/11.jpg"
predicted_class = predict_images(input_image_path)
print(f"Predicted class for the input image: {predicted_class}")

0: 640x480 2 faces, 156.4ms
Speed: 4.1ms preprocess, 156.4ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Predicted class for the input image: ['Gujjar', 'Daud']
